# Recipe Scraper Project
This portion of the project will focus on scraping recipe details from the URL pages compiled from the URL Scraper Project. This code will bring in:
* Recipe Author
* Recipe Description
* Recipe Ingredients
* Recipe Instructions
* Recipe Tags
* Recipe Ready In, Prep Time, Cook Time, Yield information

Importing libraries and database

In [1]:
import requests
from bs4 import BeautifulSoup as bs
headers = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36"}

In [2]:
import db

2024-05-15 14:18:41,588 INFO sqlalchemy.engine.Engine select version()
2024-05-15 14:18:41,589 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-05-15 14:18:41,591 INFO sqlalchemy.engine.Engine select current_schema()
2024-05-15 14:18:41,591 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-05-15 14:18:41,592 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2024-05-15 14:18:41,593 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-05-15 14:18:41,594 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-15 14:18:41,595 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2024-05-15 14:18:41,595 INFO sqlalchemy.engine.Engine [generated in 0.00030s] {'name': 'url'}
2024-05-15 14:18:41,597 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2024-05-15

In [3]:
import random

In [39]:
base_url = "https://www.recipetineats.com/"

Bringing in URLs that were imported from URL Scraper Project

In [5]:
sample_urls = random.sample([row.url for row in db.session.query(db.URL).all()],40)

2024-05-15 14:18:44,617 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-15 14:18:44,619 INFO sqlalchemy.engine.Engine SELECT url.id AS url_id, url.url AS url_url, url.recipe_name AS url_recipe_name, url.image AS url_image, url.skip AS url_skip 
FROM url
2024-05-15 14:18:44,620 INFO sqlalchemy.engine.Engine [generated in 0.00073s] {}


In [11]:
first = sample_urls[3]
first

'https://www.recipetineats.com/muesli-bars-granola-bars/'

In [12]:
def make_soup(url):
    return bs(requests.get(url, headers = headers).text, 'html.parser')

In [13]:
soup = make_soup(first)

Finding and bringing in the Author information

In [14]:
###Author
soup.find('span', 'entry-author-name').text

'Nagi'

Finding and bringing in the Description information

In [15]:
###Description
soup.find('div', 'entry-content').find('p').text

'Ultra chewy, homemade muesli bars that are cheaper, tastier and much healthier than store-bought. Tastes like a chewy caramel studded with nuts and fruit! Excellent way to clear out random leftover nuts and dried fruit. Want crunchy muesli bars? Head here!'

In [16]:
for url in sample_urls:
    if url.startswith(base_url):
        soup = make_soup(url)
        print(url)
        print(soup.find('span', 'entry-author-name').text)
        print(soup.find('div', 'entry-content').find('p').text)

https://www.recipetineats.com/muesli-bars-granola-bars/
Nagi
Ultra chewy, homemade muesli bars that are cheaper, tastier and much healthier than store-bought. Tastes like a chewy caramel studded with nuts and fruit! Excellent way to clear out random leftover nuts and dried fruit. Want crunchy muesli bars? Head here!
https://www.recipetineats.com/vegetable-curry/
Nagi
A mixed Vegetable Curry that packs a flavour punch, is loaded with veggies and made with common spices that you might even already have in your pantry! Based on a Caribbean curry, this tastes very similar to Indian curries. Make this as spicy as you want – or not!
https://www.recipetineats.com/italian-orzo-salad-risoni/
Nagi
Risoni /orzo salad, filled with an unapologetically generous amount of your new favourite garnish – crispy salami bits! There’s a good stack of bright fresh vegetables in here too, all tossed up in a tasty Italian Dressing. A new big, bold, statement pasta salad to fall in love with!
https://www.recipe

KeyboardInterrupt: 

In [17]:
test_p = soup.find('div', 'entry-content').find('p')

In [18]:
len(test_p.text)

213

In [19]:
test_p

<p>This trifle will look smashing on your Christmas table!!! Layers of custard and jelly, studded with cake and piled high with cream and fruit, it looks stunning in a trifle dish but otherwise just use a glass bowl!</p>

In [35]:
def description_helper(soup):
    description = ''
    div_tags = soup.find_all('div', class_='core-paragraph')
    for div_tag in div_tags:
        p_tags = div_tag.find_all('p')
        for p_tag in p_tags:
            if len(p_tag.text) > 0 and 'not-prose' not in p_tag.find_parent().attrs['class']:
                description += f" {p_tag.text}"
    description = description.replace("\n", '').replace("\xa0", '').strip()
    return description

In [78]:
for url in sample_urls[0:10]:
    print(url)
    soup = make_soup(url)
    print(description_helper(soup))

https://www.forksoverknives.com/recipes//recipes/amazing-grains/vegan-polenta-recipes-to-fall-in-love-with/

https://www.forksoverknives.com/recipes//recipes/vegan-breakfast/vegan-chilaquiles/
Chilaquiles originated as a way to use leftover tortillas the morning after a big meal, but this taco-inspired version would be perfect at any time of day. Here, crispy tortilla chips are smothered in a tangy homemade salsa and topped with hearty quinoa infused with the smoky, spicy flavors of Southwest seasoning. Creamy black beans add extra substance while fresh cilantro and scallions dial up the herbal aromas of this mouthwatering dish. Feel free to top with a drizzle of your favorite hot sauce, and dig in! For more vegan Mexican-inspired breakfast recipes, check out these tasty ideas:
https://www.forksoverknives.com/recipes//recipes/amazing-grains/baked-vegetable-biryani/
A true Indian classic, vegetable biryani is usually made on the stove top, but this simplified version makes for a great b

In [82]:
soup = make_soup(sample_urls[3])

In [83]:
soup.find('div', 'mb-8')

<div class="mb-8"><ul class="font-serif leading-7 text-fok-navy-500"><li class="mb-2.5 text-base lg:text-lg"><span>12.3-oz. package extra-firm silken-style tofu</span></li><li class="mb-2.5 text-base lg:text-lg"><span>¼ cup lightly packed fresh parsley</span></li><li class="mb-2.5 text-base lg:text-lg"><span>1 clove garlic</span></li><li class="mb-2.5 text-base lg:text-lg"><span>Pinch cayenne pepper</span></li><li class="mb-2.5 text-base lg:text-lg"><span>4 tablespoons lemon juice</span></li><li class="mb-2.5 text-base lg:text-lg"><span>1 clove garlic, minced</span></li><li class="mb-2.5 text-base lg:text-lg"><span>¼ teaspoon freshly ground black pepper</span></li><li class="mb-2.5 text-base lg:text-lg"><span>4 small fresh globe artichokes (8 oz. each)</span></li><li class="mb-2.5 text-base lg:text-lg"><span>2 lemon wedges</span></li></ul></div>

Finding and bringing in the Ingredient information

In [50]:
### Ingredients
for url in sample_urls[1:15]:
    if url.startswith(base_url):
        print(url)
        soup = make_soup(url)
        print([li.text for li in soup.find('div', 'mb-8').find('ol').find_all('li')])

https://www.recipetineats.com/muesli-bars-granola-bars/


AttributeError: 'NoneType' object has no attribute 'find'

In [28]:
soup.find('div', 'wprm-recipe-ingredient-group').find('h4').text

'Chocolate (read Note 1!):'

In [49]:
for url in sample_urls[1:15]:
    if url.startswith(base_url):
        print(url)
        soup = make_soup(url)    
        try:
            ingredient_groups = soup.find_all('div', class_='wprm-recipe-ingredient-group')
            if ingredient_groups:
                for ingredient_group in ingredient_groups:
                    ul_elements = ingredient_group.find_all('ul')
                    for ul_element in ul_elements:
                        # Check if ul_element has a preceding h4 element
                        h4_element = ul_element.find_previous_sibling('h4')
                        if h4_element:
                            print(h4_element.text)
                        li_items = ul_element.find_all('li')
                        for li_item in li_items:
                            print(li_item.text)
            else:
                print("No ingredient groups found")
        except Exception as e:
            print(f"Error processing URL {url}: {e}")
            continue  # Skip to the next iteration if an error occurs


https://www.recipetineats.com/muesli-bars-granola-bars/
▢ 1/2 cup (120g)   pure natural peanut butter, smooth, or other nut butter (mix oil in well, Note 1)
▢ 1/2 cup (175 g)  honey (Note 2)
Add ins (3 1/2 cups your choice, Note 3):
▢ 1 1/2 cups rolled oats
▢ 1 cup sultanas
▢ 1/4 cup desiccated coconut , unsweetened
▢ 1/2 cup almonds , very roughly chopped
▢ 1/4 cup pepitas
▢ 2 tbsp white sesame seeds
▢ 1/8 tsp salt
▢ 1/2 tsp cinnamon
https://www.recipetineats.com/vegetable-curry/
Spice Mix:
▢ 2 tbsp curry powder , mild or hot (Note 1)
▢ 1 tsp All Spice powder
▢ 1 tsp nutmeg powder (or 1/2 tsp freshly grated nutmeg)
▢ 1 1/2 tsp smoked paprika (sub with sweet or ordinary paprika)
▢ 2 tsp dried thyme leaves (or 3 tsp fresh thyme leaves)
▢ 1 tsp cumin powder
▢ 3/4 tsp cayenne pepper (adjust to taste)
▢ 1 tsp pepper (white or black)
Curry:
▢ 3 tbsp cooking oil
▢ 1 large onion , diced (brown, white, yellow)
▢ 1 large red chilli / cayenne pepper , deseeded and finely sliced
▢ 2 large garlic 

In [ ]:
<div class="wprm-recipe-ingredient-group"><h4 class="wprm-recipe-group-name wprm-recipe-ingredient-group-name wprm-block-text-bold">Spiced Lamb</h4><ul class="wprm-recipe-ingredients"><li class="wprm-recipe-ingredient" style="list-style-type: none;"><span class="wprm-checkbox-container"><input type="checkbox" id="wprm-checkbox-0" class="wprm-checkbox" aria-label="&nbsp;8 oz / 250 g  ground lamb (mince)"><label for="wprm-checkbox-0" class="wprm-checkbox-label"><span class="sr-only screen-reader-text wprm-screen-reader-text">▢ </span></label></span><span class="wprm-recipe-ingredient-amount">8 oz / 250 g </span> <span class="wprm-recipe-ingredient-name">ground lamb</span> <span class="wprm-recipe-ingredient-notes wprm-recipe-ingredient-notes-faded">(mince)</span></li><li class="wprm-recipe-ingredient" style="list-style-type: none;"><span class="wprm-checkbox-container"><input type="checkbox" id="wprm-checkbox-1" class="wprm-checkbox" aria-label="&nbsp;1/2 brown onion , finely diced"><label for="wprm-checkbox-1" class="wprm-checkbox-label"><span class="sr-only screen-reader-text wprm-screen-reader-text">▢ </span></label></span><span class="wprm-recipe-ingredient-amount">1/2</span> <span class="wprm-recipe-ingredient-name">brown onion</span> <span class="wprm-recipe-ingredient-notes wprm-recipe-ingredient-notes-faded">, finely diced</span></li><li class="wprm-recipe-ingredient" style="list-style-type: none;"><span class="wprm-checkbox-container"><input type="checkbox" id="wprm-checkbox-2" class="wprm-checkbox" aria-label="&nbsp;1 tbsp olive oil"><label for="wprm-checkbox-2" class="wprm-checkbox-label"><span class="sr-only screen-reader-text wprm-screen-reader-text">▢ </span></label></span><span class="wprm-recipe-ingredient-amount">1</span> <span class="wprm-recipe-ingredient-unit">tbsp</span> <span class="wprm-recipe-ingredient-name">olive oil</span></li><li class="wprm-recipe-ingredient" style="list-style-type: none;"><span class="wprm-checkbox-container"><input type="checkbox" id="wprm-checkbox-3" class="wprm-checkbox" aria-label="&nbsp;1/2 tsp cumin powder"><label for="wprm-checkbox-3" class="wprm-checkbox-label"><span class="sr-only screen-reader-text wprm-screen-reader-text">▢ </span></label></span><span class="wprm-recipe-ingredient-amount">1/2</span> <span class="wprm-recipe-ingredient-unit">tsp</span> <span class="wprm-recipe-ingredient-name">cumin powder</span></li><li class="wprm-recipe-ingredient" style="list-style-type: none;"><span class="wprm-checkbox-container"><input type="checkbox" id="wprm-checkbox-4" class="wprm-checkbox" aria-label="&nbsp;1/2 tsp coriander powder"><label for="wprm-checkbox-4" class="wprm-checkbox-label"><span class="sr-only screen-reader-text wprm-screen-reader-text">▢ </span></label></span><span class="wprm-recipe-ingredient-amount">1/2</span> <span class="wprm-recipe-ingredient-unit">tsp</span> <span class="wprm-recipe-ingredient-name">coriander powder</span></li><li class="wprm-recipe-ingredient" style="list-style-type: none;"><span class="wprm-checkbox-container"><input type="checkbox" id="wprm-checkbox-5" class="wprm-checkbox" aria-label="&nbsp;1/2 tsp paprika powder"><label for="wprm-checkbox-5" class="wprm-checkbox-label"><span class="sr-only screen-reader-text wprm-screen-reader-text">▢ </span></label></span><span class="wprm-recipe-ingredient-amount">1/2</span> <span class="wprm-recipe-ingredient-unit">tsp</span> <span class="wprm-recipe-ingredient-name">paprika powder</span></li><li class="wprm-recipe-ingredient" style="list-style-type: none;"><span class="wprm-checkbox-container"><input type="checkbox" id="wprm-checkbox-6" class="wprm-checkbox" aria-label="&nbsp;1/8 tsp cinnamon powder"><label for="wprm-checkbox-6" class="wprm-checkbox-label"><span class="sr-only screen-reader-text wprm-screen-reader-text">▢ </span></label></span><span class="wprm-recipe-ingredient-amount">1/8</span> <span class="wprm-recipe-ingredient-unit">tsp</span> <span class="wprm-recipe-ingredient-name">cinnamon powder</span></li><li class="wprm-recipe-ingredient" style="list-style-type: none;"><span class="wprm-checkbox-container"><input type="checkbox" id="wprm-checkbox-7" class="wprm-checkbox" aria-label="&nbsp;1/4 tsp black pepper"><label for="wprm-checkbox-7" class="wprm-checkbox-label"><span class="sr-only screen-reader-text wprm-screen-reader-text">▢ </span></label></span><span class="wprm-recipe-ingredient-amount">1/4</span> <span class="wprm-recipe-ingredient-unit">tsp</span> <span class="wprm-recipe-ingredient-name">black pepper</span></li><li class="wprm-recipe-ingredient" style="list-style-type: none;"><span class="wprm-checkbox-container"><input type="checkbox" id="wprm-checkbox-8" class="wprm-checkbox" aria-label="&nbsp;1 tsp salt"><label for="wprm-checkbox-8" class="wprm-checkbox-label"><span class="sr-only screen-reader-text wprm-screen-reader-text">▢ </span></label></span><span class="wprm-recipe-ingredient-amount">1</span> <span class="wprm-recipe-ingredient-unit">tsp</span> <span class="wprm-recipe-ingredient-name">salt</span></li></ul></div>

In [88]:
soup.find('div', 'mb-8')

<div class="mb-8"><ul class="font-serif leading-7 text-fok-navy-500"><li class="mb-2.5 text-base lg:text-lg"><span>1 cup unsweetened, unflavored plant milk</span></li><li class="mb-2.5 text-base lg:text-lg"><span>15 pitted dates</span></li><li class="mb-2.5 text-base lg:text-lg"><span>2 cups cooked brown rice</span></li><li class="mb-2.5 text-base lg:text-lg"><span>1 teaspoon pure vanilla extract</span></li><li class="mb-2.5 text-base lg:text-lg"><span>2 cups mixed fresh berries</span></li><li class="mb-2.5 text-base lg:text-lg"><span>1 cup Easy Oil-Free Granola or purchased oil-free granola</span></li></ul></div>

Finding and bringing in the Instruction information

In [120]:
### Instructions
for url in sample_urls[1:5]:
    print(url)
    soup = make_soup(url)
    print([li.text for li in soup.find('ul', 'font-serif leading-7 text-fok-navy-500 list-decimal text-lg').find('ol').find_all('li')])

https://www.forksoverknives.com/recipes//recipes/vegan-breakfast/vegan-chilaquiles/


AttributeError: 'NoneType' object has no attribute 'find_all'

In [66]:
for url in sample_urls[1:50]:
    if url.startswith(base_url):
        print(url)
        soup = make_soup(url)    
        try:
            instruction_groups = soup.find_all('div', class_='wprm-recipe-instruction-group')
            if instruction_groups:
                for instruction_group in instruction_groups:
                    ul_elements = instruction_group.find_all('ul')
                    for ul_element in ul_elements:
                        # Check if ul_element has a preceding h4 element
                        h4_element = ul_element.find_previous_sibling('h4')
                        if h4_element:
                            print(h4_element.text)
                        li_items = ul_element.find_all('li')
                        for li_item in li_items:
                            print(li_item.text)
            else:
                print("No instruction groups found")
        except Exception as e:
            print(f"Error processing URL {url}: {e}")
            continue  # Skip to the next iteration if an error occurs

https://www.recipetineats.com/muesli-bars-granola-bars/
Mix Add-ins – Using a wooden spoon, mix the add-ins in a large microwaveable bowl (just in case, Note 4).
Glue – Using a rubber spatula, mix the peanut butter and honey in a saucepan over medium heat until smooth. Once combined, turn the stove down to low. As soon as you start seeing little bubbles around the edges, cook for 5 minutes, stirring constantly scraping the base, until it is like a very thick caramel (see photos and video in post). You should be able to draw a path across the base and it holds before closing in on itself. DO NOT WALK AWAY, it will catch easily. (Note 5)
Mix – Immediately pour it all over surface of the oats etc then mix with the wooden spoon. It takes a bit of effort but it does come together eventually. Stabbing and smearing against the wall of the bowl helps. If it gets too thick, microwave for 20 seconds on high to loosen the glue then mix (then it's a breeze!).
Press – Put a square piece of paper ov

Heat oil in a large pot over medium high heat. Add onion and garlic and cook for 2 minutes until partly translucent but not browned.
Add carrots, celery and capsicum, cook for 1 minute to soften.
Add butter and melt. Then add flour and mix until incorporated, and stir constantly for 1 minute.
Add broth, mix until flour is incorporated, then add milk. Mix to combine, then add salt, Spices, chicken and peas.
Bring to simmer, mixing occasionally to stop bottom of sticking. As it heats, it will thicken – about 4 to 5 minutes. Don’t let it bubble.
Once thickened to your taste, adjust salt and pepper. Ladle into bowls and serve garnished with croutons and fresh thyme if desired.
Croutons: Spray croutons generously with oil, sprinkle with salt, then bake for 5 minutes at 180C/350F or until golden and crunchy.
https://www.recipetineats.com/italian-sausage-quiche/
Preheat oven to 180C/350F.
Lightly spray a 9"/23cm quiche flan or pie dish with oil.
Line with the pastry, trimming the excess. Use 

In [62]:
soup.find_all('div', class_='wprm-recipe-instruction-group')

[<div class="wprm-recipe-instruction-group"><ul class="wprm-recipe-instructions"><li class="wprm-recipe-instruction" id="wprm-recipe-59250-step-0-0" style="list-style-type: decimal;"><div class="wprm-recipe-instruction-text"><span style="display: block;"><strong>Spice and herb sachet: </strong>Bundle the bay leaf, thyme, black peppercorns and coriander seeds loosely in a small piece of cheesecloth to create a sachet. Tie with cooking twine to secure.</span></div></li><li class="wprm-recipe-instruction" id="wprm-recipe-59250-step-0-1" style="list-style-type: decimal;"><div class="wprm-recipe-instruction-text"><span style="display: block;"><strong>Cook onion and leek: </strong>Melt butter in large pot over medium-low heat. Add onion, leek, celery and garlic. Cook for 10 minutes until onion is soft but not golden. Don't rush this part – this creates an important flavour base so we can make this soup using water not stock.</span></div></li><li class="wprm-recipe-instruction" id="wprm-recip

In [52]:
for url in sample_urls:
    if url.startswith(base_url):
        soup = make_soup(url)
        print(url)
        print([li.text for li in soup.find('ul', 'wprm-recipe-instructions').find_all('li')])

https://www.recipetineats.com/muesli-bars-granola-bars/
['Mix Add-ins – Using a wooden spoon, mix the add-ins in a large microwaveable bowl (just in case, Note 4).', 'Glue – Using a rubber spatula, mix the peanut butter and honey in a saucepan over medium heat until smooth. Once combined, turn the stove down to low. As soon as you start seeing little bubbles around the edges, cook for 5 minutes, stirring constantly scraping the base, until it is like a very thick caramel (see photos and video in post). You should be able to draw a path across the base and it holds before closing in on itself. DO NOT WALK AWAY, it will catch easily. (Note 5)', "Mix – Immediately pour it all over surface of the oats etc then mix with the wooden spoon. It takes a bit of effort but it does come together eventually. Stabbing and smearing against the wall of the bowl helps. If it gets too thick, microwave for 20 seconds on high to loosen the glue then mix (then it's a breeze!).", 'Press – Put a square piece 

https://www.recipetineats.com/homemade-frozen-pizzas/
['Heat olive oil in a large saucepan over medium heat. Add garlic and cook for 1 minute until fragrant. Add remaining ingredients, stir, and adjust heat so it is simmering gently (about medium heat). Cook down for 40 minutes, then puree with a stick (or cool a bit then transfer to blender).\xa0', 'COOL before using (hot sauce = soggy base). (Note 4)']
https://www.recipetineats.com/celeriac-soup/
['Spice and herb sachet: Bundle the bay leaf, thyme, black peppercorns and coriander seeds loosely in a small piece of cheesecloth to create a sachet. Tie with cooking twine to secure.', "Cook onion and leek: Melt butter in large pot over medium-low heat. Add onion, leek, celery and garlic. Cook for 10 minutes until onion is soft but not golden. Don't rush this part – this creates an important flavour base so we can make this soup using water not stock.", "Cook celeriac and potato: Add celeriac and potato. Cook for 10 minutes, stirring regul

In [ ]:
<ul class="wprm-recipe-instructions"><li id="wprm-recipe-22417-step-0-0" class="wprm-recipe-instruction" style="list-style-type: decimal;"><div class="wprm-recipe-instruction-text">Heat oil in a large pot over medium high heat. Add garlic and onion, cook for 2 minutes. Add cayenne and cook until onion is translucent.</div></li><li id="wprm-recipe-22417-step-0-1" class="wprm-recipe-instruction" style="list-style-type: decimal;"><div class="wprm-recipe-instruction-text">Add Spice Mix and cook, stirring constantly, for 1 - 2 minutes until the spice is fragrant and wet. If too dry, add a tiny touch of water or oil.</div></li><li id="wprm-recipe-22417-step-0-2" class="wprm-recipe-instruction" style="list-style-type: decimal;"><div class="wprm-recipe-instruction-text">Add capsicum and zucchini, cook for 1 minute.</div></li><li id="wprm-recipe-22417-step-0-3" class="wprm-recipe-instruction" style="list-style-type: decimal;"><div class="wprm-recipe-instruction-text">Add tomato, broth, potato and cauliflower. Mix well, bring to simmer, cover, lower heat to medium.</div></li><li id="wprm-recipe-22417-step-0-4" class="wprm-recipe-instruction" style="list-style-type: decimal;"><div class="wprm-recipe-instruction-text"><span style="display: block;">Simmer for 15 minutes or until potato is pretty soft, then remove lid and simmer for another 10 minutes+ until sauce is slightly thickened and potato is soft.</span></div></li><li id="wprm-recipe-22417-step-0-5" class="wprm-recipe-instruction" style="list-style-type: decimal;"><div class="wprm-recipe-instruction-text">Add peas, stir, cook for 2 minutes. Add coriander and spinach, stir until wilted.</div></li><li id="wprm-recipe-22417-step-0-6" class="wprm-recipe-instruction" style="list-style-type: decimal;"><div class="wprm-recipe-instruction-text"><span style="display: block;">Season to taste until salt and pepper. Serve over <a href="https://www.recipetineats.com/how-to-cook-basmati-rice/">basmati rice</a> or <a href="https://www.recipetineats.com/how-to-cook-rice/">plain white rice</a>. Garnish with yoghurt and more coriander.</span></div></li></ul>

Finding and bringing in the Tag information

In [70]:
### Tags
for url in sample_urls[1:50]:
    if url.startswith(base_url):
        print(url)
        soup = make_soup(url)
        try:
            print([li.text for li in soup.find('div', 'wprm-recipe-block-container wprm-recipe-block-container-separate wprm-block-text-normal wprm-recipe-tag-container wprm-recipe-keyword-container').find_all('span')])
        except:
            print('no')

https://www.recipetineats.com/muesli-bars-granola-bars/
['Keywords: ', 'granola bars, muesli bars']
https://www.recipetineats.com/vegetable-curry/
no
https://www.recipetineats.com/italian-orzo-salad-risoni/
['Keywords: ', 'chicken pasta salad, orzo salad, risoni salad']
https://www.recipetineats.com/10-classic-chinese-dishes-1-amazing-sauce/
no
https://www.recipetineats.com/rocky-road-recipe/
['Keywords: ', 'rocky road, rocky road recipe']
https://www.recipetineats.com/golden-gaytime-popcorn-copycat/
['Keywords: ', 'caramel corn, Caramel popcorn, food gift ideas, golden gaytime popcorn']
https://www.recipetineats.com/christmas-trifle/
['Keywords: ', 'Christmas dessert, christmas trifle, trifle']
https://www.recipetineats.com/homemade-cream-chicken-soup/
['Keywords: ', 'Cream of chicken soup']
https://www.recipetineats.com/italian-sausage-quiche/
no
https://www.recipetineats.com/charlottes-hazelnut-olive-oil-cake/
['Keywords: ', 'dairy free cake, Gluten Free Cake, olive oil cake']
https

In [ ]:
<div class="wprm-recipe-block-container wprm-recipe-block-container-separate wprm-block-text-normal wprm-recipe-tag-container wprm-recipe-keyword-container" style=""><span class="wprm-recipe-details-label wprm-block-text-normal wprm-recipe-tag-label wprm-recipe-keyword-label">Keywords: </span><span class="wprm-recipe-keyword wprm-block-text-normal">Homemade frozen pizza</span></div>

In [105]:
soup = make_soup(sample_urls[3])
soup.find('div','box-post-control mb-6').find_all('li')

[<li class="mr-3 flex w-full items-center text-13px tracking-wide lg:w-auto"><i class="icon-time mr-2 text-21px text-fok-teal-600 lg:text-[27px]"></i><span><span>Prep-time:<!-- --> <time datetime="PT10M">10 MINUTES</time> <!-- -->/<!-- --> </span><span>Ready In:<!-- --> <time datetime="PT30M">30 MINUTES</time></span></span></li>,
 <li class="mr-3 flex w-full items-center tracking-wide lg:w-auto"><i class="icon-serving mr-2 text-21px text-fok-teal-600 lg:text-[27px]"></i><span>Makes 4 artichokes</span></li>,
 <li class="mr-3 flex w-full items-center tracking-wide lg:w-auto"><a class="popup-print print-recipe-link flex items-center font-sans text-[13px] uppercase !text-black hover:!text-fok-teal-500" href="#"><i class="icon-print text mr-2 text-21px text-fok-teal-600 lg:text-[27px]"></i><span>Print/save recipe</span></a></li>]

In [ ]:
<div class="box-post-control mb-6"><ul class="add-info my-0 flex flex-col gap-2.5 !pl-0 font-sans text-[13px] uppercase text-black lg:flex-row"><li class="mr-3 flex w-full items-center text-13px tracking-wide lg:w-auto"><i class="icon-time mr-2 text-21px text-fok-teal-600 lg:text-[27px]"></i><span><span>Prep-time:<!-- --> <time datetime="PT30M">30 MINUTES</time> <!-- -->/<!-- --> </span><span>Ready In:<!-- --> <time datetime="PT40M">40 MINUTES</time></span></span></li><li class="mr-3 flex w-full items-center tracking-wide lg:w-auto"><i class="icon-serving mr-2 text-21px text-fok-teal-600 lg:text-[27px]"></i><span>Makes 7½ cups</span></li><li class="mr-3 flex w-full items-center tracking-wide lg:w-auto"><a class="popup-print print-recipe-link flex items-center font-sans text-[13px] uppercase !text-black hover:!text-fok-teal-500" href="#"><i class="icon-print text mr-2 text-21px text-fok-teal-600 lg:text-[27px]"></i><span>Print/save recipe</span></a></li></ul></div>

Finding and bringing in the Ready In, Prep Time, Cook Time

In [83]:
[span.text for span in soup.find('div','wprm-recipe-block-container wprm-recipe-block-container-separate wprm-block-text-normal wprm-recipe-time-container wprm-recipe-prep-time-container').find('span')]

AttributeError: 'NoneType' object has no attribute 'find'

In [ ]:
<div class="wprm-recipe-block-container wprm-recipe-block-container-separate wprm-block-text-normal wprm-recipe-time-container wprm-recipe-prep-time-container" style=""><span class="wprm-recipe-details-label wprm-block-text-bold wprm-recipe-time-label wprm-recipe-prep-time-label"><svg class="svg-inline--fa fa-clock fa-w-16" aria-hidden="true" data-prefix="fas" data-icon="clock" role="img" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 512 512" data-fa-i2svg=""><path fill="currentColor" d="M256 8C119 8 8 119 8 256s111 248 248 248 248-111 248-248S393 8 256 8zm57.1 350.1L224.9 294c-3.1-2.3-4.9-5.9-4.9-9.7V116c0-6.6 5.4-12 12-12h48c6.6 0 12 5.4 12 12v137.7l63.5 46.2c5.4 3.9 6.5 11.4 2.6 16.8l-28.2 38.8c-3.9 5.3-11.4 6.5-16.8 2.6z"></path></svg> Prep: </span><span class="wprm-recipe-time wprm-block-text-normal"><span class="wprm-recipe-details wprm-recipe-details-minutes wprm-recipe-prep_time wprm-recipe-prep_time-minutes">20<span class="sr-only screen-reader-text wprm-screen-reader-text"> minutes</span></span> <span class="wprm-recipe-details-unit wprm-recipe-details-minutes wprm-recipe-prep_time-unit wprm-recipe-prep_timeunit-minutes" aria-hidden="true">mins</span></span></div>

In [104]:
for url in sample_urls[1:15]:
    if url.startswith(base_url):
        soup = make_soup(url)
        spans = soup.find('div', 'wprm-entry-info').find_all('span')
        for i in range(len(spans) - 1):
            span = spans[i]
            next_span = spans[i + 1]
            if "Total" in span.text:
                print(f"{span.text}: {next_span.text}")
            elif "Prep" in span.text:
                print(url)
                print(f"{span.text}: {next_span.text}")
            elif "Cook:" in span.text:
                print(f"{span.text}: {next_span.text}")

https://www.recipetineats.com/muesli-bars-granola-bars/
 Prep: : 15 minutes mins
 Cook: : 5 minutes mins
https://www.recipetineats.com/vegetable-curry/
 Prep: : 15 minutes mins
 Cook: : 25 minutes mins
 Total: : 40 minutes mins
https://www.recipetineats.com/italian-orzo-salad-risoni/
 Prep: : 15 minutes mins
 Cook: : 15 minutes mins
https://www.recipetineats.com/rocky-road-recipe/
 Prep: : 15 minutes mins
 Total: : 3 hours hrs 15 minutes mins


In [84]:
base_url = "https://www.recipetineats.com/"
for url in sample_urls[1:15]:
    if url.startswith(base_url):
        soup = make_soup(url)
        spans = soup.find('div','wprm-entry-info').find_all('span')
        for span in spans[1:]:
            if "Total" in span.text:
                print(span.text)
            elif "Prep" in span.text:
                print(span.text)
            elif "Cook:" in span.text:
                print(span.text)


 Prep: 
 Cook: 
 Prep: 
 Cook: 
 Total: 
 Prep: 
 Cook: 
 Prep: 
 Total: 


In [111]:
test = "https://www.forksoverknives.com/recipes/vegan-breakfast/vegan-chilaquiles/"

In [112]:
soup = make_soup(test)

In [113]:
with open('testhtml.html', 'w+') as f:
    f.write(soup.prettify())

UnicodeEncodeError: 'charmap' codec can't encode character '\u2193' in position 56751: character maps to <undefined>